## Detect Peaks

In [ ]:
def detect_peaks(x, mph=None, mpd=1, threshold=0, edge='rising', kpsh=False, valley=False):

    """
    
    Detect peaks in data based on their amplitude and other features.

    Parameters
    ----------
    x : 1D array_like
        data.
        
    mph : {None, number}, default=None
        detect peaks that are greater than minimum peak height.
        
    mpd : int, default=1
        detect peaks that are at least separated by minimum peak distance (in number of data).
        
    threshold : int, default=0
        detect peaks (valleys) that are greater (smaller) than `threshold in relation to their immediate neighbors.
        
    edge : str, default=rising
        for a flat peak, keep only the rising edge ('rising'), only the falling edge ('falling'), both edges ('both'), or don't detect a flat peak (None).
        
    kpsh : bool, default=False
        keep peaks with same height even if they are closer than `mpd`.
        
    valley : bool, default=False
        if True (1), detect valleys (local minima) instead of peaks.

    Returns
    ---------
    ind : 1D array_like
        indeces of the peaks in `x`.

    Modified from 
   ----------------
    .. [1] http://nbviewer.ipython.org/github/demotu/BMC/blob/master/notebooks/DetectPeaks.ipynb
    

    """

    x = np.atleast_1d(x).astype('float64')
    if x.size < 3:
        return np.array([], dtype=int)
    if valley:
        x = -x
    # find indices of all peaks
    dx = x[1:] - x[:-1]
    # handle NaN's
    indnan = np.where(np.isnan(x))[0]
    if indnan.size:
        x[indnan] = np.inf
        dx[np.where(np.isnan(dx))[0]] = np.inf
    ine, ire, ife = np.array([[], [], []], dtype=int)
    if not edge:
        ine = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) > 0))[0]
    else:
        if edge.lower() in ['rising', 'both']:
            ire = np.where((np.hstack((dx, 0)) <= 0) & (np.hstack((0, dx)) > 0))[0]
        if edge.lower() in ['falling', 'both']:
            ife = np.where((np.hstack((dx, 0)) < 0) & (np.hstack((0, dx)) >= 0))[0]
    ind = np.unique(np.hstack((ine, ire, ife)))
    # handle NaN's
    if ind.size and indnan.size:
        # NaN's and values close to NaN's cannot be peaks
        ind = ind[np.in1d(ind, np.unique(np.hstack((indnan, indnan-1, indnan+1))), invert=True)]
    # first and last values of x cannot be peaks
    if ind.size and ind[0] == 0:
        ind = ind[1:]
    if ind.size and ind[-1] == x.size-1:
        ind = ind[:-1]
    # remove peaks < minimum peak height
    if ind.size and mph is not None:
        ind = ind[x[ind] >= mph]
    # remove peaks - neighbors < threshold
    if ind.size and threshold > 0:
        dx = np.min(np.vstack([x[ind]-x[ind-1], x[ind]-x[ind+1]]), axis=0)
        ind = np.delete(ind, np.where(dx < threshold)[0])
    # detect small peaks closer than minimum peak distance
    if ind.size and mpd > 1:
        ind = ind[np.argsort(x[ind])][::-1]  # sort ind by peak height
        idel = np.zeros(ind.size, dtype=bool)
        for i in range(ind.size):
            if not idel[i]:
                # keep peaks with the same height if kpsh is True
                idel = idel | (ind >= ind[i] - mpd) & (ind <= ind[i] + mpd)                     & (x[ind[i]] > x[ind] if kpsh else True)
                idel[i] = 0  # Keep current peak
        # remove the small peaks and sort back the indices by their occurrence
        ind = np.sort(ind[~idel])

    return ind



## Reading Predicted Probability

In [ ]:
import matplotlib.pyplot as plt
import numpy as np



yh2 = np.load('pred_SS_mean_all_FondationEQCCT.npy', allow_pickle=True)
yh2_std = np.load('pred_SS_std_all_FoundationEQCC.npy', allow_pickle=True)
sst = np.load('pall_FoundationEQCCT.npy', allow_pickle=True)



thre=0.1
S_PICKall=[]
Spickall=[]
Pproball = []
serrorall=[]
S_uncertaintyall = []

for i in range(0,len(yh2)):
    
    yh3 = yh2[i]
    yh3_std = yh2_std[i]
    
    
    sS_arr = detect_peaks(yh3, mph=thre, mpd=1)

    S_PICKS = []
    pick_errors = []
    #print(ssst)
    S_uncertainty = None
    if len(sS_arr) > 0:
        S_uncertainty = None  

        for pick in range(len(sS_arr)):        
            sauto = sS_arr[pick]

            #if  sauto:
            #    S_uncertainty = np.round(yh3_std[int(sauto)], 3)

            if sauto: 
                S_prob = np.round(yh3[int(sauto)], 3) 
                S_PICKS.append([sauto,S_prob, S_uncertainty]) 
                
    S_uncertaintyall.append(S_uncertainty)
    
    so=[]
    si=[]
    S_PICKS = np.array(S_PICKS)
    S_PICKall.append(S_PICKS)
    for ij in S_PICKS:
        so.append(ij[1])
        si.append(ij[0])
    if so:
        so = np.array(so)
        inds = np.argmax(so)
        swave = si[inds]
        #serrorall.append(int(ssst[i]- swave))  
        Spickall.append(int(swave))
        Pproball.append(int(np.max(so)))
    else:
        serrorall.append(None)
        Spickall.append(None)
        Pproball.append(None)


Spickall = np.array(Spickall)
serrorall = np.array(serrorall)


## Metrics

In [ ]:
Spick = Spickall

swave = []
swavetp=[]
swavetn=[]
swavefp=[]
swavefn=[]
difts=[]
iqq=[]
fals=[]
cc = 0
for iq in range(0,len(sst)):
    
    if (Spick[iq]!=None) and (sst[iq]!=None):
        swavetp.append(sst[iq]-Spick[iq])
    elif (Spick[iq]==None) and (sst[iq]!=None):
        swavefn.append(iq)
        
    elif (Spick[iq]==None) and (sst[iq]==None):
        swavetn.append(iq)
    
    elif (Spick[iq]!=None) and (sst[iq]==None):
        swavefp.append(iq)
        fals.append(iq)




samp = 50
difts = np.array(swavetp)
TP = len(np.where(np.abs(difts)<=samp)[0])
TN = len(swavetn) 
FP = len(swavefp) 
FN = len(swavefn) + len(np.where(np.abs(difts)>samp)[0])
P = TP /(TP+FP)
R = TP / (TP+FN)
F1 = 2 * (P*R) / (P+R)

#print(TP + TN + FP + FN, TP, TN, FP, FN,len(swavefn) , len(np.where(np.abs(difts)>samp)[0]))
print('Total number of tested events is:',len(yh2))
print('TP is:',TP)
print('FP is:',FP)
print('TN is:',TN)
print('FN is:',FN)
print('Precision is:',P)
print('Recall is :',R)
print("F1-score is:",F1)
print('Number of missing Events is:',len(swavefn))
print('Number of Picked Event with error >0.5:',len(np.where(np.abs(difts)>samp)[0]))
a0 = np.where(np.abs(difts)<=samp)[0]
diftss = difts[a0]/100
print('std is:',np.std(diftss))
print('MAE is:',np.mean(np.abs(diftss)))
#np.save('difpp.npy',diftsp)


## Histogram error

In [ ]:
font = {'weight' : 'bold',
        'size'   : 14}
plt.rc('font', **font)

fig = plt.figure()
ax = fig.add_subplot(1,1,1)
plt.hist(diftss, bins=15)
plt.xlim([-1,1])
plt.ylim([0,50000])
plt.xlabel("S-picking Error(s) ",fontsize='large', fontweight='bold')
plt.ylabel("Count ",fontsize='large', fontweight='bold')
plt.title("Foundation EQCCT",fontsize='large', fontweight='bold')

plt.grid()
ax.set_axisbelow(True)
plt.text(-0.9,4000, 'MAE=' + str(np.round(np.mean(np.abs(diftss)),3)) )
plt.text(-0.9,1000, '$\sigma$=' + str(np.round(np.std((diftss)),3)) )
plt.show()
#fig.savefig('P_Cal_WC.png', bbox_inches='tight',transparent=True, dpi =100)
